### 対象のライブラリ

`io`は ストリームを扱う標準ライブラリ

- 特徴

  - テキストなどの一時処理が手軽
  - with 文の併用で auto close できるため安全性が高い

- 主な使用用途

  - 単純なファイル操作
  - ファイルの入出力テストを IO を使用して、インメモリでファイルを扱う

- 注意

### 資料

[ドキュメント](https://docs.python.org/ja/3.13/library/io.html)


In [1]:
import io

## StringIO(インメモリテキストストリーム)

[StringIO-ドキュメント](https://docs.python.org/ja/3.13/library/io.html#text-i-o)


In [4]:
f = io.StringIO("This is StringIO")
print(f.read())
f.close()

This is StringIO


In [12]:
# 基本的にはwithと一緒に使う
with io.StringIO("This is StringIO") as f:
    print(f.tell())  # 現在のオフセットを返す
    print(f.read())

0
This is StringIO


In [14]:
# Offsetを指定して、文字列を書き込む
with io.StringIO("This is StringIO") as f:
    f.seek(0, io.SEEK_END)
    f.write("\nHello World")
    print(f.getvalue())


This is StringIO
Hello World


In [2]:
# GPTのポエムを読み込む
text = """
    朝焼けに染まる道を
    昨日の夢を引きずって歩く
    ひとしずくの希望が
    ポケットの中で光ってる
    今日も、始めてみようか。
"""

with io.StringIO(text) as f:
    print(f.getvalue())  # getvalueで保持している内容全てを文字列で返す
    for tx in f:
        print(tx)


    朝焼けに染まる道を
    昨日の夢を引きずって歩く
    ひとしずくの希望が
    ポケットの中で光ってる
    今日も、始めてみようか。



    朝焼けに染まる道を

    昨日の夢を引きずって歩く

    ひとしずくの希望が

    ポケットの中で光ってる

    今日も、始めてみようか。



## BytesIO(インメモリバイナリストリーム)

[BytesIO-ドキュメント](https://docs.python.org/ja/3.13/library/io.html#binary-i-o)


In [11]:
# 基本的にはStringIOと同様のメソッドが使用できる
with io.BytesIO(b"abcdefg") as f:
    print(f.tell())
    print(f.read(5)) # b'abcde'

0
b'abcde'


In [21]:
with io.BytesIO(b"abcdefg") as f:
    f.seek(0, io.SEEK_END)
    f.write(b'test')
    print(f.getvalue())

b'abcdefgtest'


In [35]:
stream = io.BytesIO(b'abcdefg')
buf = stream.getbuffer()
buf[2:4] = b'55'
print(stream.getvalue())
del buf
stream.close()  # close呼ばない場合エラーは起きないので、del bufは不要

b'ab55efg'


### del buf がないと、buf がずっと参照されている状態になってしまい close でエラーが発生する

- export を increment して close の際にチェックしているよう。

[参考：confusing BufferError: Existing exports of data: object cannot be re-sized #85269](https://github.com/python/cpython/issues/85269)

[\_io/bytesio.c#L1080](https://github.com/python/cpython/blob/d87e7f35297d34755026173d84a38eedfbed78de/Modules/_io/bytesio.c#L1080)

[\_io/bytesio.c#L51](https://github.com/python/cpython/blob/d87e7f35297d34755026173d84a38eedfbed78de/Modules/_io/bytesio.c#L51)

```python
---------------------------------------------------------------------------
BufferError                               Traceback (most recent call last)
Cell In[33], line 5
      3 buf[2:4] = b'55'
      4 print(stream.getvalue())
----> 5 stream.close()

BufferError: Existing exports of data: object cannot be re-sized
```


In [32]:
with io.BytesIO(b"abcdefg") as f:
    buf = f.getbuffer()
    buf[2:4] = b'55'
    print(f.getvalue())
    del buf

b'ab55efg'


## io でユニットテスト

- file オブジェクトの代わり
- 標準出力などをキャプチャしておく


In [36]:
from unittest.mock import patch

In [42]:
"""
print_hogeが出力していることを検証したい
stdoutをioと差し替えることで、正しく出力が行われているかチェックすることができる
"""
def print_hoge():
    print("hoge")

@patch('sys.stdout', new_callable=io.StringIO)  # 標準出力をioと差し替える
def test_print_hoge(mocked_object):
    print_hoge()
    assert mocked_object.getvalue() == "hoge\n"

test_print_hoge()

## io を使用した周辺知識


In [48]:
import contextlib, logging

In [47]:
# 一時的に別ファイルまたは、file-lileにリダイレクトすることができる
# stdoutをioへリダイレクトさせて、捕捉する
f = io.StringIO()
with contextlib.redirect_stdout(f):
    print("hello")

print(f.getvalue())

hello



In [51]:

f = io.StringIO()
with contextlib.redirect_stderr(f):
    logging.warning("What's happend")

print(f.getvalue())